- input embeddings = token embeddings + segment embeddings + position embeddings
- input embeddings [lookup table]
    - token embeddings: 30522*768
    - segment embeddings: 2*768
    - position embeddings: 512*768
- 后处理
    - layer norm
    - dropout

In [1]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn

d:\anaconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
test_sentence = "Hello, my dog is cute"
input_sequence = tokenizer(test_sentence, return_tensors="pt")
input_ids = input_sequence["input_ids"]
token_type_ids = input_sequence["token_type_ids"]

In [4]:
print(input_ids)
print(token_type_ids)

tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0]])


In [19]:
print(input_ids.shape)
print(token_type_ids.shape)
print(pos_ids.shape)
print(torch.arange(input_ids.shape[1]).shape)

torch.Size([1, 8])
torch.Size([1, 8])
torch.Size([1, 8])
torch.Size([8])


In [18]:
pos_ids = torch.arange(input_ids.shape[1]).unsqueeze(0)
pos_ids

tensor([[0, 1, 2, 3, 4, 5, 6, 7]])

In [6]:
# 1. token embeddings
token_embeddings = model.embeddings.word_embeddings(input_ids)
# 2. segment type embeddings
segment_embeddings = model.embeddings.token_type_embeddings(token_type_ids)
# 3. position type embeddings
position_embeddings = model.embeddings.position_embeddings(pos_ids)
print(token_embeddings.shape)
print(segment_embeddings.shape)
print(position_embeddings.shape)

torch.Size([1, 8, 768])
torch.Size([1, 8, 768])
torch.Size([1, 8, 768])


In [7]:
# 4. sum of all embeddings
input_embeddings = token_embeddings + segment_embeddings + position_embeddings

In [8]:
# 5. layer norm
input_embeddings = model.embeddings.LayerNorm(input_embeddings)
# 6. dropout
input_embeddings = model.embeddings.dropout(input_embeddings)